In [1]:
import sys
import os
import nbimporter
import copy

# 外皮情報LV4からLV5へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 用途別の外皮、内壁の情報が得られている状態より、以下を実施する。  
　　床フラグの付与   
　　応答係数の算出  
　　複数の壁体、部位の応答係数を集約  
　　気象データを用いた開口部透過・吸収日射量の算出（未）  

## 2. Convert

### 2.1 Common Element

#### Outline

'Common'要素はそのまま維持する。

#### Function

In [2]:
def convert_common(d):
    return d['Common']

### 2.2 Floor Flag

#### Outline

以下に該当する部位を、床と判定する。  
　方位が'Bottom'である外皮の部位  
　土間床  
　床下空間上の床  
　室上の床  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [3]:
def set_floor_flag(d):
    
    # 方位が'Bottom'である外皮の部位
    for x in ['Walls', 'Windows', 'Doors'] :
        if (x in d) == True :
            for y in d[x] :
                y['IsFloor'] = True if y['Direction'] == 'Bottom' else False    
    
    # 土間床
    if ('EarthFloors' in d) == True :
        for x in d['EarthFloors'] :
            x['IsFloor'] = True    
    
    # 床下空間上の床、室上の床
    if ('InnerWalls' in d) == True :
        for x in d['InnerWalls'] :
            x['IsFloor'] = True if (x['Type'] == 'GroundFloor' or x['Type'] == 'InnerFloor') else False 
    
    return d

#### Example

In [4]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_main', 'Space': 'Main'},
        {'Area': 1.26, 'Direction': 'NW', 'name': 'DoorNW_other', 'Space': 'Other'},
        {'Area': 0.63, 'Direction': 'NW', 'name': 'DoorNW_nonliving', 'Space': 'Nonliving'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_main', 'Space': 'Main'},
        {'Area': 1.08, 'Direction': 'NE', 'name': 'DoorNE_other', 'Space': 'Other'},
        {'Area': 0.54, 'Direction': 'NE', 'name': 'DoorNE_nonliving', 'Space': 'Nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'Direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'Space': 'Underfloor'},
        {'Direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'Space': 'Underfloor'}
    ],
    'EarthFloors': [
        {'Area': 30.0, 'Name': 'Main', 'Space': 'Main'},
        {'Area': 5.0, 'Name': 'Entrance', 'Space': 'Nonliving'}
    ],
    'Walls': [
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 33.9, 'Direction': 'top', 'name': 'Ceiling_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 16.95, 'Direction': 'top', 'name': 'Ceiling_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Ceiling'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_main', 'Space': 'Main', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 20.315, 'Direction': 'SW', 'name': 'Wall_SW_other', 'Space': 'Other', 'structure': 'wood', 'Type': 'Wall'},
        {'Area': 10.1575, 'Direction': 'SW', 'name': 'Wall_SW_nonliving', 'Space': 'Nonliving', 'structure': 'wood', 'Type': 'Wall'}
    ],
    'Windows': [
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 15.125, 'Direction': 'SW', 'name': 'WindowSW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 7.5625, 'Direction': 'SW', 'name': 'WindowSW_nonliving',  'Space': 'Nonliving', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_main', 'Space': 'Main', 'type': 'single'},
        {'Area': 1.585, 'Direction': 'NW', 'name': 'WindowNW_other', 'Space': 'Other', 'type': 'single'},
        {'Area': 0.7925, 'Direction': 'NW', 'name': 'WindowNW_nonliving', 'Space': 'Nonliving', 'type': 'single'}
    ],
    'InnerWalls': [
        {'Area': 0.0, 'Direction': 'Horizontal', 'Name': 'GroundFloor_Main', 'Spaces': ['Main', 'Underfloor'], 'Type': 'GroundFloor'},
        {'Area': 0.0, 'Direction': 'Horizontal', 'Name': 'GroundFloor_Other', 'Spaces': ['Other', 'Underfloor'], 'Type': 'GroundFloor'},
        {'Area': 0.0, 'Direction': 'Horizontal', 'Name': 'GroundFloor_Nonliving', 'Spaces': ['Nonliving', 'Underfloor'], 'Type': 'GroundFloor'},
        {'Area': 0.0, 'Direction': 'Horizontal', 'Name': 'InnerFloor_Main', 'NextSpace': 'Other', 'Space': 'Main', 'Type': 'InnerFloor'},
        {'Area': 0.0, 'Direction': 'Horizontal', 'Name': 'InnerFloor_Main',  'NextSpace': 'Nonliving', 'Space': 'Main', 'Type': 'InnerFloor'},
        {'Area': 30.0, 'Direction': 'Horizontal', 'Name': 'InnerFloor_Other', 'NextSpace': 'Main',  'Space': 'Other', 'Type': 'InnerFloor'},
        {'Area': 30.0, 'Direction': 'Horizontal', 'Name': 'InnerFloor_Other',  'NextSpace': 'Nonliving', 'Space': 'Other', 'Type': 'InnerFloor'},
        {'Area': 8.3, 'Direction': 'Horizontal', 'Name': 'InnerFloor_Nonliving', 'NextSpace': 'Main', 'Space': 'Nonliving', 'Type': 'InnerFloor'},
        {'Area': 16.7, 'Direction': 'Horizontal', 'Name': 'InnerFloor_Nonliving', 'NextSpace': 'Other', 'Space': 'Nonliving', 'Type': 'InnerFloor'},
        {'Area': 21.2, 'Direction': 'Horizontal', 'Name': 'InnerCeiling_Main', 'NextSpace': 'Other', 'Space': 'Main', 'Type': 'InnerCeiling'},
        {'Area': 8.8, 'Direction': 'Horizontal', 'Name': 'InnerCeiling_Main', 'NextSpace': 'Nonliving', 'Space': 'Main','Type': 'InnerCeiling'},
        {'Area': 0.0, 'Direction': 'Horizontal', 'Name': 'InnerCeiling_Other', 'NextSpace': 'Main', 'Space': 'Other','Type': 'InnerCeiling'},
        {'Area': 60.0, 'Direction': 'Horizontal', 'Name': 'InnerCeiling_Other', 'NextSpace': 'Nonliving', 'Space': 'Other', 'Type': 'InnerCeiling'},
        {'Area': 0.0, 'Direction': 'Horizontal', 'Name': 'InnerCeiling_Nonliving', 'NextSpace': 'Main', 'Space': 'Nonliving', 'Type': 'InnerCeiling'},
        {'Area': 30.0, 'Direction': 'Horizontal', 'Name': 'InnerCeiling_Nonliving', 'NextSpace': 'Other', 'Space': 'Nonliving', 'Type': 'InnerCeiling'},
        {'Area': 18.0, 'Direction': 'Vertical', 'Name': 'Innerwall_main to other', 'Spaces': ['Main', 'Other'], 'Type': 'InnerWall'},
        {'Area': 19.0, 'Direction': 'Vertical', 'Name': 'Innerwall_main to nonliving', 'Spaces': ['Main', 'Nonliving'], 'Type': 'InnerWall'},
        {'Area': 30.2, 'Direction': 'Vertical', 'Name': 'Innerwall_other to nonliving', 'Spaces': ['Other', 'Nonliving'], 'Type': 'InnerWall'}
    ]
}

set_floor_flag(d)

{'Common': {'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Doors': [{'Area': 0.63,
   'Direction': 'NW',
   'IsFloor': False,
   'Space': 'Main',
   'name': 'DoorNW_main'},
  {'Area': 1.26,
   'Direction': 'NW',
   'IsFloor': False,
   'Space': 'Other',
   'name': 'DoorNW_other'},
  {'Area': 0.63,
   'Direction': 'NW',
   'IsFloor': False,
   'Space': 'Nonliving',
   'name': 'DoorNW_nonliving'},
  {'Area': 0.54,
   'Direction': 'NE',
   'IsFloor': False,
   'Space': 'Main',
   'name': 'DoorNE_main'},
  {'Area': 1.08,
   'Direction': 'NE',
   'IsFloor': False,
   'Space': 'Other',
   'name': 'DoorNE_other'},
  {'Area': 0.54,
   'Direction': 'NE',
   'IsFloor': False,
   'Space': 'Nonliving',
   'name': 'DoorNE_nonliving'}],
 'EarthFloors': [{'Area': 30.0,
   'IsFloor': True,
   'Name': 'Main',
   'Space': 'Main'},
  {'Area': 5.0, 'IsFloor': True, 'Name': 'Entrance', 'Space': 'Nonliving'}],
 'EarthfloorPerimeters': [{

### 2.3 Ro, Ri

以下に基づき、表面熱伝達抵抗を与える。

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  
　　　　　付録 A 住宅の平均熱貫流率算出に用いる建材等の熱物性値等   
　　　　　　表 3.1 表面熱伝達抵抗   
　　　　　　表 3.2 表面熱伝達抵抗（界壁・界床の場合） 

<div style="text-align: center;"> 表3.1 表面熱伝達抵抗 </div>

| 部位 | 熱的境界内側（室内側）の<br>表面熱伝達抵抗[㎡K/W] | 熱的境界外側（外気側）の表面熱伝達抵抗[㎡K/W]<br>外気に直接接する場合 | 熱的境界外側（外気側）の表面熱伝達抵抗[㎡K/W]<br>左記以外の場合 |
|---|---|---|---|
| 屋根 | 0.09 | 0.04 | 0.09（通気層等） |
| 天井 | 0.09 | - | 0.09（小屋裏等） |
| 外壁 | 0.11 | 0.04 | 0.11（通気層等） |
| 床 | 0.15 | 0.04 | 0.15(床裏等) |

<div style="text-align: center;"> 表3.2 表面熱伝達抵抗（界壁・界床の場合） </div>

| 部位 | 対象住戸の室内側表面熱伝達抵抗[㎡K/W] | 隣接住戸の室内側表面熱伝達抵抗[㎡K/W] |
|---|---|---|
| 界壁 | 0.11 | 0.11 |
| 上階側界床 | 0.09 | 0.09 |
| 下階側界床 | 0.15 | 0.15 |

In [5]:
def get_Ro_Ri(Type, NextSpace):
    
    if NextSpace == 'OutsideAir' :
        Ro = {
            'Roof'            : 0.040,
            'Wall'            : 0.040,
            'Floor'           : 0.040
        }[Type]
    else :
        Ro = {
            'Roof'            : 0.090,
            'Ceiling'         : 0.090,
            'Wall'            : 0.110,
            'Floor'           : 0.150,
            'BoundaryCeiling' : 0.090,
            'BoundaryWall'    : 0.110,
            'BoundaryFloor'   : 0.150
        }[Type]
        
    Ri = {
        'Roof' : 0.090,
        'Ceiling' : 0.090, 
        'Wall' : 0.110, 
        'Floor' : 0.150, 
        'BoundaryCeiling' : 0.090,
        'BoundaryWall' : 0.110,
        'BoundaryFloor'   : 0.150
    }[Type]
    
    return Ro, Ri

### 2.4 Responce Factor

#### Outline

以下に基づき、一般部位の熱貫流率を算出する。

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  

#### 詳細計算法

$$
\displaystyle
U_{i}= \sum_{k}{a_{i,k} \times U_{i,k}}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$a_{i,k}$: 一般部位$i$の部位$k$の面積比率, -  
$U_{i,k}$: 一般部位$i$の部位$k$の熱貫流率, W/(m<sup>2</sup>・K)  

In [6]:
def calc_UA_from_details(d_wall):
    
    # 平均熱貫流率の算出
    Ro, Ri = get_Ro_Ri(d_wall['Type'], d_wall['NextSpace'])
    UA = sum(d_wall['Parts'][x]['AreaRatio']/(Ro + sum(y['Thick']/y['Cond'] for y in d_wall['Parts'][x]['Layers']) + Ri) \
             for x in d_wall['Parts'])
    
    return UA

In [7]:
d = {
    'Walls' : [
        {
            'NextSpace' : 'ClosedBackFloor',
            'Structure' : 'Wood', 
            'Type' : 'Floor',
            'InputMethodWood' : 'InputAllDetails',
            'Parts' :  {
                'Insulation' : {'AreaRatio' : 0.80, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'Heatbridge' : {'AreaRatio' : 0.20, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]} 
         }
        },
        {
            'NextSpace' : 'OutsideAir',
            'Structure' : 'Wood', 
            'Type' : 'Roof',
            'InputMethodWood' : 'InputAllDetails',
            'Parts' :  {
                'FrameInsulRafterandadd' : {'AreaRatio' : 0.75, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameInsulrafterHeatbridgeadd' : {'AreaRatio' : 0.08, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameInsuladdHeatbridgemember' : {'AreaRatio' : 0.12, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameHeatbridgeMemberandadd' : {'AreaRatio' : 0.05, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}
         }
        },
        {
            'Structure' : 'Wood', 
            'NextSpace' : 'OutsideAir',
            'Type' : 'Wall',
            'InputMethodWood' : 'InputAllDetails',
             'Parts' :  {
                'FrameInsulFilledandadd' : {'AreaRatio' : 0.76, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameInsulfilledHeatbridgeadd' : {'AreaRatio' : 0.01, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameInsuladdHeatbridgemember' : {'AreaRatio' : 0.00, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameInsuladdHeatbridgelintel' : {'AreaRatio' : 0.02, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameHeatbridgeMemberandadd' : {'AreaRatio' : 0.20, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'FrameHeatbridgeLintelandadd' : {'AreaRatio' : 0.01, 'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]} 
         }
        }
    ]
}

def get_UA_from_details(d):
    for x in d['Walls'] :
        if x['InputMethodWood'] == 'InputAllDetails' :
            x['UA'] = calc_UA_from_details(x)
    return d

print(get_UA_from_details(d))

{'Walls': [{'NextSpace': 'ClosedBackFloor', 'Structure': 'Wood', 'Type': 'Floor', 'InputMethodWood': 'InputAllDetails', 'Parts': {'Insulation': {'AreaRatio': 0.8, 'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}, 'Heatbridge': {'AreaRatio': 0.2, 'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}, 'UA': 2.6666666666666665}, {'NextSpace': 'OutsideAir', 'Structure': 'Wood', 'Type': 'Roof', 'InputMethodWood': 'InputAllDetails', 'Parts': {'FrameInsulRafterandadd': {'AreaRatio': 0.75, 'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}, 'FrameInsulrafterHeatbridgeadd': {'AreaRatio': 0.08, 'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}, 'FrameInsuladdHeatbridgemember': {'AreaRatio': 0.12, 'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}, 'FrameHeatbridgeMemberandadd': {'AreaRatio': 0.05, 'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}, 'UA

#### 面積比率法（充填断熱する場合又は充填断熱し付加断熱する場合）（簡略計算方法①） 

$$
\displaystyle
U_{i}= \sum_{k}{a_{i,k} \times U_{i,k}}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$a_{i,k}$: 一般部位$i$の部位$k$の面積比率, -  
$U_{i,k}$: 一般部位$i$の部位$k$の熱貫流率, W/(m<sup>2</sup>・K)  

#### 木造における床の面積比率

<div style="text-align: center;"> 表3-1 木造における床の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・床梁工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（大引間に断熱）  | 0.85 | 0.15 |
| 軸組構法・束立大引工法（根太間及び大引間に断熱）  |  |  |
| 軸組構法・剛床工法  | 0.85 | 0.15 |
| 軸組構法・床梁土台同面工法（根太間に断熱）  | 0.70 | 0.30 |
| 枠組壁工法（根太間に断熱）  | 0.87 | 0.13 |

<div style="text-align: center;"> 表3-2 軸組構法の束立大引工法において根太間及び大引間に断熱する場合の床の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（根太間断熱材＋大引間断熱材） | 断熱部分＋熱橋部分<br>（根太間断熱材＋大引材等） | 断熱部分＋熱橋部分<br>（根太材＋大引間断熱材） | 熱橋部分<br>（根太材＋大引材等） |
|---|---|---|---|---|
|  軸組構法の束立大引工法において<br>根太間及び大引間に断熱する場合 | 0.72 | 0.12 | 0.13 | 0.03 |

In [8]:
def get_area_ratio_wood_floor(FloorConstructionMethod):
    # FloorConstructionMethod: 床の工法種類
    # 'FrameBeamInsuljoist'（軸組構法・床梁工法（根太間に断熱））/'FrameFootingInsuljoist'（軸組構法・束立大引工法（根太間に断熱））/
    # 'FrameFootingInsulsleeper'（軸組構法・束立大引工法（大引間に断熱））/
    # 'FrameFootingInsuljoistSleeper'（軸組構法・束立大引工法（根太間及び大引間に断熱））/
    # 'FrameRigidfloor'（軸組構法・剛床工法）/'FrameBeambaseInsuljoist'（軸組構法・床梁土台同面工法（根太間に断熱））/
    # 'WallInsuljoist'（枠組壁工法（根太間に断熱））
    
    if FloorConstructionMethod == 'FrameBeamInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsulsleeper' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameFootingInsuljoistSleeper' :
        area_ratio_wood_floor = {
            'InsulJoitandsleeper' : 0.72,
            'InsuljoistHeatbridgesleeper' : 0.12,
            'InsulsleeperHeatbridgejoist' : 0.13,
            'HeatbridgeJoistandsleeper' : 0.03
        }
    elif FloorConstructionMethod == 'FrameRigidfloor' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameBeambaseInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.70, 'Heatbridge' : 0.30}
    elif FloorConstructionMethod == 'WallInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.87, 'Heatbridge' : 0.13}
    else :
        raise ValueError
    
    return area_ratio_wood_floor

#### 木造における外壁の面積比率

<div style="text-align: center;"> 表4-1 木造における外壁（界壁）の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・柱、間柱間に断熱 | 0.83 | 0.17 |
| 軸組構法・柱、間柱間に断熱し付加断熱 |  |  |
| 枠組壁工法・たて枠間に断熱 | 0.77 | 0.23 |
| 枠組壁工法・たて枠間に断熱＋付加断熱 |  |  |

<div style="text-align: center;"> 表4-2 軸組構法において柱・間柱間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（柱・間柱間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（柱・間柱間断熱材＋付加断熱層内熱橋部分） | 断熱部分＋熱橋部分<br>（構造部材等＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部分） |
|---|---|---|
| 付加断熱層内熱橋<br>部分が「横下地」 | 0.75 | 0.08 | 0.12 | 0.05 |
| 付加断熱層内熱橋<br>部分が「縦下地」 | 0.79 | 0.04 | 0.04 | 0.13 |

<div style="text-align: center;"> 表4-3 枠組壁工法においてたて枠間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（充填断熱材＋付加断熱材） | 断熱部分＋熱橋部<br>（充填断熱材＋付加断熱層内熱橋部） | 断熱部分＋熱橋部<br>（構造部材等＋付加断熱材） | 断熱部分＋熱橋部<br>（まぐさ＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部） | 熱橋部分<br>（まぐさ＋付加断熱層内熱橋部） |
|---|---|---|
| 付加断熱層内熱橋部分が「横下地」 | 0.69 | 0.08 | 0.14 | 0.02 | 0.06 | 0.01 |
| 付加断熱層内熱橋部分が「縦下地」 | 0.76 | 0.01 |  | 0.02 | 0.20 | 0.01 |

In [9]:
def get_area_ratio_wood_wall(WallConstructionMethod):
    # WallConstructionMethod: 床の工法種類
    # 'FrameInsulcolumn'（軸組構法・柱、間柱間に断熱）/
    # 'FrameInsuladdBackhorizontal'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'FrameInsuladdBackvertical'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「縦下地」））/
    # 'WallInsuljamb'（枠組壁工法・たて枠間に断熱）/
    # 'WallInsuladdBackhorizontal'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'WallInsuladdBackvertical'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「縦下地」））
    
    if WallConstructionMethod == 'FrameInsulcolumn' :
        area_ratio_wood_wall = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    elif WallConstructionMethod == 'FrameInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'FrameInsulColumnandadd' : 0.75,
            'FrameInsulcolumnHeatbridgeadd' : 0.08,
            'FrameInsuladdHeatbridgemember' : 0.12,
            'FrameHeatbridgeMemberandadd' : 0.05
        }
    elif WallConstructionMethod == 'FrameInsuladdBackvertical' :
        area_ratio_wood_wall = {
            'FrameInsulColumnandadd' : 0.79,
            'FrameInsulcolumnHeatbridgeadd' : 0.04,
            'FrameInsuladdHeatbridgemember' : 0.04,
            'FrameHeatbridgeMemberandadd' : 0.13
        }
    elif WallConstructionMethod == 'WallInsuljamb' :
        area_ratio_wood_wall = {'Insulation' : 0.77, 'Heatbridge' : 0.23}
    elif WallConstructionMethod == 'WallInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'WallInsulFilledandadd' : 0.69,
            'WallInsulfilledHeatbridgeadd' : 0.08,
            'WallInsuladdHeatbridgemember' : 0.14,
            'WallInsuladdHeatbridgelintel' : 0.02,
            'WallHeatbridgeMemberandadd' : 0.06,
            'WallHeatbridgeLintelandadd' : 0.01
        }
    elif WallConstructionMethod == 'WallInsuladdBackvertical' :
        area_ratio_wood_wall = {        
            'WallInsulFilledandadd' : 0.76,
            'WallInsulfilledHeatbridgeadd' : 0.01,
            'WallInsuladdHeatbridgemember' : 0.00,
            'WallInsuladdHeatbridgelintel' : 0.02,
            'WallHeatbridgeMemberandadd' : 0.20,
            'WallHeatbridgeLintelandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_wall

#### 木造における天井の面積比率

<div style="text-align: center;"> 表5 木造における天井の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 桁・梁間に断熱する場合 | 0.87 | 0.13 |

In [10]:
def get_area_ratio_wood_ceiling(CeilingConstructionMethod):
    # CeilingConstructionMethod: 天井の工法種類
    # 'Insulbeam'（桁・梁間に断熱）
    
    if CeilingConstructionMethod == 'Insulbeam' :
        area_ratio_wood_wall = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    else :
        raise ValueError

    return area_ratio_wood_ceiling

#### 木造における屋根の面積比率

<div style="text-align: center;"> 表6-1 木造における屋根の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| たるき間に断熱する場合  | 0.86 | 0.14 |

<div style="text-align: center;"> 表6-2 木造においてたるき間に断熱し付加断熱（横下地）する場合の屋根の面積比率  </div>

| 工法の種類等 | 断熱部分<br>（たる木間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（たる木間断熱材＋付加断熱層内熱橋部（下地たる木）） | 断熱部分＋熱橋部分<br>（構造部材＋付加断熱材） | 熱橋部分<br>（構造部材＋付加断熱層内熱橋部（下地たる木）） |
|---|---|---|---|---|
| たるき間に断熱し付加断熱（横下地）する場合  | 0.79 | 0.08 | 0.12 | 0.01 |

In [11]:
def get_area_ratio_wood_roof(RoofConstructionMethod):
    # RoofConstructionMethod : 屋根の工法種類
    # 'Insulrafter'（たるき間に断熱）/'InsuladdBackhorizontal'（たるき間に断熱＋付加断熱（横下地）））
    
    if RoofConstructionMethod == 'Insulrafter' :
        area_ratio_wood_roof = {'Insulation' : 0.86, 'Heatbridge' : 0.14}
    elif RoofConstructionMethod == 'InsuladdBackhorizontal' :
        area_ratio_wood_roof = {
            'InsulRafterandadd' : 0.79,
            'InsulrafterHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.12,
            'HeatbridgeMemberandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_roof

In [12]:
def get_area_ratio_wood(d_wall):
    
    if d_wall['Type'] == 'Roof' :
        return get_area_ratio_wood_roof(d_wall['RoofConstructionMethod'])
    elif d_wall['Type'] == 'Ceiling' or d_wall['Type'] == 'BoundaryCeiling' :
        return get_area_ratio_wood_ceiling(d_wall['CeilingConstructionMethod'])        
    elif d_wall['Type'] == 'Wall' or d_wall['Type'] == 'BoundaryWall' :
        return get_area_ratio_wood_wall(d_wall['WallConstructionMethod'])            
    elif d_wall['Type'] == 'Floor' :
        return get_area_ratio_wood_floor(d_wall['FloorConstructionMethod'])    
    else :
        raise ValueError        

In [13]:
def calc_UA_from_layers(d_wall):
    
    # 面積比率の取得
    area_ratio =  get_area_ratio_wood(d_wall)
    for x in d_wall['Parts'].keys() :
        d_wall['Parts'][x]['AreaRatio'] = area_ratio[x]

    # 平均熱貫流率の算出
    Ro, Ri = get_Ro_Ri(d_wall['Type'], d_wall['NextSpace'])
    UA = sum(d_wall['Parts'][x]['AreaRatio']/(Ro + sum(y['Thick']/y['Cond'] for y in d_wall['Parts'][x]['Layers']) + Ri) \
             for x in d_wall['Parts'])
    
    return UA

#### Example

In [14]:
d = {
    'Walls' : [
        {
            'Area': 10,
            'Direction' : 'Bottom',
            'Name' : 'Floor',
            'Space' : 'Main',
            'NextSpace' : 'ClosedBackFloor',
            'Structure' : 'Wood', 
            'Type' : 'Floor',
            'InputMethodWood' : 'InputAllLayers',
            'FloorConstructionMethod' : 'FrameFootingInsuljoist',
            'Parts' :  {
                'Insulation' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'Heatbridge' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]} 
         }
        },
        {
            'Area': 20,
            'Direction' : 'Top',
            'Name' : 'Roof',
            'Space' : 'Main',
            'NextSpace' : 'OutsideAir',
            'Structure' : 'Wood', 
            'Type' : 'Roof',
            'InputMethodWood' : 'InputAllLayers',
            'RoofConstructionMethod' : 'InsuladdBackhorizontal',
            'Parts' :  {
                'InsulRafterandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'InsulrafterHeatbridgeadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'InsuladdHeatbridgemember' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'HeatbridgeMemberandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}
         }
        },
        {
            'Area': 30,
            'Direction' : 'North',
            'Name' : 'Wall',
            'Space' : 'Main',
            'Structure' : 'Wood', 
            'NextSpace' : 'OutsideAir',
            'Type' : 'Wall',
            'InputMethodWood' : 'InputAllLayers',
            'WallConstructionMethod' : 'WallInsuladdBackvertical',
            'Parts' :  {
                'WallInsulFilledandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallInsulfilledHeatbridgeadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallInsuladdHeatbridgemember' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallInsuladdHeatbridgelintel' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallHeatbridgeMemberandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallHeatbridgeLintelandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]} 
         }
        }
    ]
}

def get_UA_from_layers(d):
    for x in d['Walls'] :
        if x['InputMethodWood'] == 'InputAllLayers' :
            x['UA'] = calc_UA_from_layers(x)
    return d

print(get_UA_from_layers(d))

{'Walls': [{'Area': 10, 'Direction': 'Bottom', 'Name': 'Floor', 'Space': 'Main', 'NextSpace': 'ClosedBackFloor', 'Structure': 'Wood', 'Type': 'Floor', 'InputMethodWood': 'InputAllLayers', 'FloorConstructionMethod': 'FrameFootingInsuljoist', 'Parts': {'Insulation': {'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}], 'AreaRatio': 0.8}, 'Heatbridge': {'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}], 'AreaRatio': 0.2}}, 'UA': 2.6666666666666665}, {'Area': 20, 'Direction': 'Top', 'Name': 'Roof', 'Space': 'Main', 'NextSpace': 'OutsideAir', 'Structure': 'Wood', 'Type': 'Roof', 'InputMethodWood': 'InputAllLayers', 'RoofConstructionMethod': 'InsuladdBackhorizontal', 'Parts': {'InsulRafterandadd': {'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}], 'AreaRatio': 0.79}, 'InsulrafterHeatbridgeadd': {'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}], 'AreaRatio': 0.08}, 'InsuladdHeatbridgemember': {'Lay

#### 熱貫流率補正法（簡略計算方法②）

<div style="text-align: center;"> 表7 木造における一般部位の断熱工法等に応じた補正熱貫流率 </div>

| 部位 | 断熱工法等 | 軸組構法補正熱貫流率 | 枠組壁構法補正熱貫流率 |
|---|---|---|---|
| 床 |  | 0.13 | 0.08 |
| 上階側界床 |  | 0.13 | 0.08 |
| 下階側界床 |  | 0.13 | 0.08 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）する場合 | 0.09 | 0.13 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）し付加断熱する場合 | 0.09 | 0.13 |

| 部位 | 断熱工法等 | 補正熱貫流率 |
|---|---|---|
| 外壁・界壁 | 土壁で外張断熱の場合 | 0.04 |
| 外壁・界壁 | 土壁以外で外張断熱の場合 | 0.02 |
| 天井 | 桁・梁間に断熱する場合 | 0.05 |
| 屋根 | 充填断熱（たるき間に断熱）する場合 | 0.11 |
| 屋根 | 充填断熱（たるき間に断熱）し付加断熱する場合 | 0.11 |
| 屋根 | 外張断熱工法 | 0.02 |

In [15]:
def get_Ur_wood(d_wall):
    # Type : 部位種別
    # ConstructionMethod : 工法種類
    # 'FrameInsulcolumn'（軸組構法）/'WallInsul'（枠組壁構法）
    
    if d_wall['Type'] == 'Floor' or d_wall['Type'] == 'BoundaryCeiling' or d_wall['Type'] == 'BoundaryFloor' :
        if d_wall['ConstructionMethod'] == 'FrameInsulcolumn' :
            Ur = 0.13
        elif d_wall['ConstructionMethod'] == 'WallInsul' :
            Ur = 0.08
        else :
            raise ValueError

    # WallInsulationMethod : 壁の断熱種類
    # 'InsulColumn'（充填断熱（柱・間柱間に断熱））/'InsulColumnandadd'（充填断熱（柱・間柱間に断熱）+付加断熱）/
    # 'OuterInsulMudWall'（土壁で外張断熱）/'OuterInsul'（土壁以外で外張断熱）    
    elif d_wall['Type'] == 'Wall' or d_wall['Type'] == 'BoundaryWall' :
        if d_wall['WallInsulationMethod'] == 'InsulColumn' or d_wall['RoofInsulationMethod'] == 'InsulColumnandadd' :
            if d_wall['ConstructionMethod'] == 'FrameInsulcolumn' :
                Ur = 0.09
            elif d_wall['ConstructionMethod'] == 'WallInsul' :
                Ur = 0.13
            else :
                raise ValueError
        elif d_wall['WallInsulationMethod'] == 'OuterInsulMudWall' :
            Ur = 0.04
        elif d_wall['WallInsulationMethod'] == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    elif d_wall['Type'] == 'Ceiling' :
        Ur = 0.05

    # RoofInsulationMethod : 屋根の断熱種類
    # 'InsulRafter'（充填断熱（たるき間に断熱））/'InsulRafterandadd'（充填断熱（たるき間に断熱）+付加断熱）/'OuterInsul'（外張断熱工法）
    elif d_wall['Type'] == 'Roof' :
        if d_wall['RoofInsulationMethod'] == 'InsulRafter' or d_wall['RoofInsulationMethod'] == 'InsulRafterandadd' :
            Ur = 0.11
        elif d_wall['RoofInsulationMethod'] == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    else :
        raise ValueError
    
    return Ur

<div style="text-align: center;"> 表8 鉄骨造における一般部位の熱橋部分（柱及び梁以外）の仕様に応じた補正熱貫流率 </div>

| 「外装材＋断熱補強材」の熱抵抗(m<sup>2</sup>･K/W) | 補正熱貫流率ܷ |
|---|---|
| 1.7以上 | 0.00 |
| 1.7未満1.5以上 | 0.10 |
| 1.5未満1.3以上 | 0.13 |
| 1.3未満1.1以上 | 0.14 |
| 1.1未満0.9以上 | 0.18 |
| 0.9未満0.7以上 | 0.22 |
| 0.7未満0.5以上 | 0.40 |
| 0.5未満0.3以上 | 0.45 |
| 0.3未満0.1以上 | 0.60 |
| 0.1未満 | 0.70 |

In [16]:
def get_Ur_steel(Rc):
    # Rer : 「外装材＋断熱補強材」の熱抵抗, m2K/W
    
    if Rc >= 1.7 :
        Ur = 0.00
    elif Rc >= 1.5 :
        Ur = 0.10
    elif Rc >= 1.3 :
        Ur = 0.13    
    elif Rc >= 1.1 :
        Ur = 0.14    
    elif Rc >= 0.9 :
        Ur = 0.18    
    elif Rc >= 0.7 :
        Ur = 0.22    
    elif Rc >= 0.5 :
        Ur = 0.40    
    elif Rc >= 0.3 :
        Ur = 0.45    
    elif Rc >= 0.1 :
        Ur = 0.60  
    elif Rc < 0.1 :
        Ur = 0.70
    else :
        raise ValueError

    return Ur

$$
\displaystyle
U_{i}= \frac{1}{R_{g,i}} + U_{r,i}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$R_{g,i}$: 一般部位$i$の断熱部分の熱抵抗, m<sup>2</sup>K/W  
$U_{r,i}$: 一般部位$i$の補正熱貫流率, W/(m<sup>2</sup>・K)  

In [17]:
def calc_UA_from_Ur(d_wall):
    
    # 断熱部分の熱抵抗, m2K/W
    Ro, Ri = get_Ro_Ri(d_wall['Type'], d_wall['NextSpace'])
    Rg = Ro + sum(x['Thick']/x['Cond'] for x in d_wall['Parts']['Insulation']['Layers']) + Ri
    
    # 補正熱貫流率
    if d_wall['Structure'] == 'Wood' :
        Ur = get_Ur_wood(d_wall)
    elif d_wall['Structure'] == 'Steel' :
        Ur = get_Ur_steel(d_wall['RCladding'])
    else :
        raise ValueError
    
    return 1/Rg + Ur

#### Example

In [18]:
d = {
    'Walls' : [
        {
            'Area': 10,
            'Direction' : 'Bottom',
            'Name' : 'Floor',
            'Space' : 'Main',
            'NextSpace' : 'OutsideAir',
            'Structure' : 'Wood', 
            'Type' : 'Floor',
            'InputMethodWood' : 'InputUR',
            'ConstructionMethod' : 'WallInsul',
            'Parts' :  {'Insulation' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}
        },
        {
            'Area': 20,
            'Direction' : 'Top',
            'Name' : 'Roof',
            'Space' : 'Main',
            'NextSpace' : 'OutsideAir',
            'Structure' : 'Wood', 
            'Type' : 'Roof',
            'InputMethodWood' : 'InputUR',
            'ConstructionMethod' : 'FrameInsulcolumn',
            'RoofInsulationMethod' : 'InsulRafterandadd',
            'Parts' :  {'Insulation' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}
        },
        {
            'Area': 30,
            'Direction' : 'North',
            'Name' : 'Wall',
            'Space' : 'Main',
            'NextSpace' : 'OutsideAir',
            'Structure' : 'Steel', 
            'Type' : 'Wall',
            'InputMethodWood' : 'InputUR',
            'RCladding' : 1.0,
            'Parts' :  {'Insulation' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}
        }
    ]
}

def get_UA_from_Ur(d):
    for x in d['Walls'] :
        if x['InputMethodWood'] == 'InputUR' :
            x['UA'] =  calc_UA_from_Ur(x)
    return d

print(get_UA_from_Ur(d))

{'Walls': [{'Area': 10, 'Direction': 'Bottom', 'Name': 'Floor', 'Space': 'Main', 'NextSpace': 'OutsideAir', 'Structure': 'Wood', 'Type': 'Floor', 'InputMethodWood': 'InputUR', 'ConstructionMethod': 'WallInsul', 'Parts': {'Insulation': {'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}, 'UA': 3.853584905660377}, {'Area': 20, 'Direction': 'Top', 'Name': 'Roof', 'Space': 'Main', 'NextSpace': 'OutsideAir', 'Structure': 'Wood', 'Type': 'Roof', 'InputMethodWood': 'InputUR', 'ConstructionMethod': 'FrameInsulcolumn', 'RoofInsulationMethod': 'InsulRafterandadd', 'Parts': {'Insulation': {'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}, 'UA': 4.988048780487805}, {'Area': 30, 'Direction': 'North', 'Name': 'Wall', 'Space': 'Main', 'NextSpace': 'OutsideAir', 'Structure': 'Steel', 'Type': 'Wall', 'InputMethodWood': 'InputUR', 'RCladding': 1.0, 'Parts': {'Insulation': {'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}}, 'UA

#### RC造

$$
\displaystyle
U_{i}= U_{i,k}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$U_{i,k}$: 一般部位$i$の部分$k$の熱貫流率, W/(m<sup>2</sup>・K)  

In [19]:
def calc_UA_from_layer(d_wall):
    
    # 断熱部分の熱抵抗, m2K/W
    Ro, Ri = get_Ro_Ri(d_wall['Type'], d_wall['NextSpace'])
    Rg = Ro + sum(x['Thick']/x['Cond'] for x in d_wall['Parts']['Insulation']['Layers']) + Ri
    
    return 1/Rg

#### 応答係数算出用層構成の作成

一般部位の熱貫流率$U_A$に一致するよう、一般部位の部分の層構成を調整する。  
－$U_A$に一致するまで、Layersの順番に厚を減らす  
－$U_A$に一致するまで、Layersの逆順に厚を減らす  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの順番）  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順）  

In [20]:
# UAをもとに層構成を調整
# Layersの順番に厚を減らす
def get_layers_from_UA_acending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in layers :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                x['Thick'] = max(0, (1/UA - Ro - Ri - (sum(y['Thick']/y['Cond'] for y in layers)-x['Thick']/x['Cond']))*x['Cond'])
    
    return layers

In [21]:
# UAをもとに層構成を調整
# Layersの逆順に厚を減らす
def get_layers_from_UA_descending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in reversed(layers) :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                x['Thick'] = max(0, (1/UA - Ro - Ri - (sum(y['Thick']/y['Cond'] for y in layers)-x['Thick']/x['Cond']))*x['Cond'])
    
    return layers

In [22]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
def get_layers_from_UA_ins_acending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        l_copy = copy.deepcopy(l)
        sort_list = []
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(0,len(l_copy)) :
                if cond_temp == l_copy[y]['Cond'] :
                    sort_list.append(y)
                    l_copy[y]['Cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['Cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                R = Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers) - layers[x]['Thick']/layers[x]['Cond']
                layers[x]['Thick'] = max(0, (1/UA - R)*layers[x]['Cond'])
    
    return layers

In [23]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
def get_layers_from_UA_ins_descending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        l_copy = copy.deepcopy(l)
        sort_list = []
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(len(l_copy),0,-1) :
                if cond_temp == l_copy[y-1]['Cond'] :
                    sort_list.append(y-1)
                    l_copy[y-1]['Cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['Cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
    if UA < 1/(Ro + Ri + sum(x['Thick']/x['Cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers)) :
                R = Ro + Ri + sum(y['Thick']/y['Cond'] for y in layers) - layers[x]['Thick']/layers[x]['Cond']
                layers[x]['Thick'] = max(0, (1/UA - R)*layers[x]['Cond'])
    
    return layers

#### 応答係数算出用の層構成の決定

応答係数算出用の層構成を決定する。  
－簡略計算法①の断熱部分  
－面積比率が最大の部分  

In [24]:
def get_part_gneral_wood_floor(FloorConstructionMethod):
    # FloorConstructionMethod: 床の工法種類
    # 'FrameBeamInsuljoist'（軸組構法・床梁工法（根太間に断熱））/'FrameFootingInsuljoist'（軸組構法・束立大引工法（根太間に断熱））/
    # 'FrameFootingInsulsleeper'（軸組構法・束立大引工法（大引間に断熱））/
    # 'FrameFootingInsuljoistSleeper'（軸組構法・束立大引工法（根太間及び大引間に断熱））/
    # 'FrameRigidfloor'（軸組構法・剛床工法）/'FrameBeambaseInsuljoist'（軸組構法・床梁土台同面工法（根太間に断熱））/
    # 'WallInsuljoist'（枠組壁工法（根太間に断熱））
    
    if FloorConstructionMethod == 'FrameBeamInsuljoist' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameFootingInsuljoist' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameFootingInsulsleeper' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameFootingInsuljoistSleeper' :
        part_gneral = 'InsulJoitandsleeper'
    elif FloorConstructionMethod == 'FrameRigidfloor' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameBeambaseInsuljoist' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'WallInsuljoist' :
        part_gneral = 'Insulation'
    else :
        raise ValueError
    
    return part_gneral

In [25]:
def get_part_gneral_wood_wall(WallConstructionMethod):
    # WallConstructionMethod: 床の工法種類
    # 'FrameInsulcolumn'（軸組構法・柱、間柱間に断熱）/
    # 'FrameInsuladdBackhorizontal'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'FrameInsuladdBackvertical'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「縦下地」））/
    # 'WallInsuljamb'（枠組壁工法・たて枠間に断熱）/
    # 'WallInsuladdBackhorizontal'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'WallInsuladdBackvertical'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「縦下地」））
    
    if WallConstructionMethod == 'FrameInsulcolumn' :
        part_gneral = 'Insulation'
    elif WallConstructionMethod == 'FrameInsuladdBackhorizontal' :
        part_gneral = 'FrameInsulColumnandadd'
    elif WallConstructionMethod == 'FrameInsuladdBackvertical' :
        part_gneral = 'FrameInsulColumnandadd'
    elif WallConstructionMethod == 'WallInsuljamb' :
        part_gneral = 'Insulation'
    elif WallConstructionMethod == 'WallInsuladdBackhorizontal' :
        part_gneral = 'WallInsulFilledandadd'
    elif WallConstructionMethod == 'WallInsuladdBackvertical' :
        part_gneral = 'WallInsulFilledandadd'
    else :
        raise ValueError
    
    return part_gneral

In [26]:
def get_part_gneral_wood_ceiling(CeilingConstructionMethod):
    # CeilingConstructionMethod: 天井の工法種類
    # 'Insulbeam'（桁・梁間に断熱）
    
    if CeilingConstructionMethod == 'Insulbeam' :
        part_gneral = 'Insulation'
    else :
        raise ValueError

    return part_gneral

In [27]:
def get_part_gneral_wood_roof(RoofConstructionMethod):
    # RoofConstructionMethod : 屋根の工法種類
    # 'Insulrafter'（たるき間に断熱）/'InsuladdBackhorizontal'（たるき間に断熱＋付加断熱（横下地）））
    
    if RoofConstructionMethod == 'Insulrafter' :
        part_gneral = 'Insulation'
    elif RoofConstructionMethod == 'InsuladdBackhorizontal' :
        part_gneral = 'InsulRafterandadd'
    else :
        raise ValueError
    
    return part_gneral

#### 応答係数算出

層構成より応答係数を算出する。  

In [28]:
def get_responsce_factor_from_layers(name, Ro, Ri, Layers) :

    import nbimporter
    import heatload_calc.Wall as mf
    
    layers =[mf.Layer( x['Name'], x['Cond'], x['SpecH'], x['Thick'] ) for x in Layers]
    layers.append(mf.Layer( 'OutHeatTrans', 1/Ro, 0.0, 0.00 ))

    hr = 5.0 
    wall = mf.Wall(name, 0.90, 0.80, 1/Ri - hr, hr, layers)

    rf =mf.ResponseFactor('wall', 3600, 50, wall)

    return {'RFT0' : rf.RFT0(), 'RFT1' : rf.RFT1() ,'RFA0' : rf.RFA0(), 'RFA1' : rf.RFA1()}

#### 一般部位構造種別・壁体仕様入力方法(木造)に応じた応答係数の算出

一般部位構造種別・壁体仕様入力方法(木造)に応じて、一般部位のU値の算出、応答係数算出用の層構成の決定、応答係数の算出を行う。  

In [29]:
def get_responce_factor(d_common, d_wall):  
    
    d_wall_copy = copy.deepcopy(d_wall)     
    
    # 表面熱伝達抵抗の取得
    Ro, Ri = get_Ro_Ri(d_wall['Type'], d_wall['NextSpace'])
    
    
    # 簡易外皮入力
    if d_common['IsSimplifiedInput']== True :

         #層構成を取得
        import nbimporter
        import ConvertUValueToSpec as mf       
        layers = mf.convert_u_value_to_spec( d_wall['Structure'], d_wall['Type'], d_wall['UA'] )
        d_wall_copy['Parts'] = {'General':
                                    {'Area': d_wall['Area'],
                                     'Layers' : [ { 'Name' : l.name, 'Cond' : l.cond, 'Thick' : l.thick, 'SpecH' : l.spech } for l in layers ]}
                               }
        
    # 木造 詳細計算法
    elif d_wall['Structure'] == 'Wood' and d_wall['InputMethodWood'] == 'InputAllDetails' :
        pass

    # 木造 簡略計算法①
    elif (d_wall['Structure'] == 'Wood' and d_wall['InputMethodWood'] == 'InputAllLayers') :
        area_ratio = get_area_ratio_wood(d_wall)
        for x in d_wall_copy['Parts'] :
            d_wall_copy['Parts'][x]['Area'] = area_ratio[x]*d_wall['Area'] 

    # 木造 簡略計算法②、鉄骨造
    elif (d_wall['Structure'] == 'Wood' and d_wall['InputMethodWood'] == 'InputUA') or d_wall['Structure'] == 'Steel' :
        UA = calc_UA_from_Ur(d_wall)
        d_wall_copy['Parts']['Insulation']['Layers'] = get_layers_from_UA_ins_descending(d_wall['Parts']['Insulation']['Layers'], UA, Ro, Ri)
        d_wall_copy['Parts']['Insulation']['Area'] = d_wall['Area'] 
    
    # RC造
    elif d_wall['Structure'] == 'RC' :
        UA = calc_UA_from_layer(d_wall)
        
    # その他
    elif d_wall['Structure'] == 'Other' :
        raise ValueError
        
    else :
        raise ValueError
  

    # 応答係数の取得
    for x in d_wall_copy['Parts'] :
        d_wall_copy['Parts'][x]['ResponceFactor'] = get_responsce_factor_from_layers(d_wall['Name'], Ro, Ri, d_wall_copy['Parts'][x]['Layers'])
        
    return d_wall_copy

#### 応答係数の係数の集約

簡易熱負荷計算でひとつの部位とみなされる部位・部分の応答係数を集約する。    
どの部位・部分を集約するかは既知であるとする。ここでは、暫定的に、同じ用途、同じ方位の部位を集約するものとする。

In [30]:
def  integrate_responce_factor(d):
    
    # 面積加重平均
    def get_area_weighted_average(area_old, area_add, value_old, value_add):
            return (area_old*value_old + area_add*value_add)/(area_old + area_add)

    d_responce_factor = {'Main':{}, 'Other':{}, 'Nonliving':{}}
    for x in d :
        for y in x['Parts'].keys() :
            
            if (x['Space'] in d_responce_factor) == True :       
                # 他の部位・部分の応答係数に集約       
                if (x['Direction'] in d_responce_factor[x['Space']]) == True :              

                    area_old = d_responce_factor[x['Space']][x['Direction']]['Area']
                    area_add = x['Parts'][y]['Area']

                    responce_factor_old = copy.deepcopy(d_responce_factor[x['Space']][x['Direction']]['ResponceFactor'])
                    responce_factor_add = x['Parts'][y]['ResponceFactor']

                    d_responce_factor[x['Space']][x['Direction']]['Area'] = area_old + area_add

                    # 応答係数の面積加重平均値の取得
                    for z in ['RFA', 'RFT']:
                        d_responce_factor[x['Space']][x['Direction']]['ResponceFactor'][z+'0'] \
                            = get_area_weighted_average(area_old, area_add, \
                                                        responce_factor_old[z+'0'],responce_factor_add[z+'0'])
                        for y in range(0,len(responce_factor_old[z+'1'])) :
                            d_responce_factor[x['Space']][x['Direction']]['ResponceFactor'][z+'1'][y] \
                                = get_area_weighted_average(area_old, area_add, \
                                                        responce_factor_old[z+'1'][y],responce_factor_add[z+'1'][y])
            
                # 当該部位の応答係数を使用
                else :
                    d_responce_factor[x['Space']][x['Direction']] = {'Area':x['Parts'][y]['Area'], 
                                                                     'ResponceFactor':x['Parts'][y]['ResponceFactor']}            
           
            # 当該部位の応答係数を使用
            else :
                d_responce_factor[x['Space']][x['Direction']] = {'Area':x['Parts'][y]['Area'], 
                                                                 'ResponceFactor':x['Parts'][y]['ResponceFactor']}
                
    return  d_responce_factor

#### Example

In [31]:
d = {
    'Common': {'Region': 6,
               'MainOccupantRoomFloorArea': 30.0,
               'OtherOccupantRoomFloorArea': 60.0,
               'TotalFloorArea': 120.0,
               'IsSimplifiedInput': True
              },    
    'Walls' : [
        {
            'Area': 20,
            'Direction' : 'Bottom',
            'Name' : 'Floor',
            'Space' : 'Main',
            'NextSpace' : 'ClosedBackFloor',
            'Structure' : 'Wood', 
            'Type' : 'Floor',
            'UA': 1.0,
            'InputMethodWood' : 'InputAllDetails',
            'Parts' :  {
                'Insulation' : {'Area': 10,
                                'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.024}]},
                'Heatbridge' : {'Area': 5,
                                'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]} 
         }
        },
        {
            'Area': 20,
            'Direction' : 'Bottom',
            'Name' : 'Floor',
            'Space' : 'Main',
            'NextSpace' : 'ClosedBackFloor',
            'Structure' : 'Wood', 
            'Type' : 'Floor',
            'UA': 1.0,
            'InputMethodWood' : 'InputUA',
            'ConstructionMethod' : 'FrameInsulcolumn',
            'Parts' :  {'Insulation' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.024}]}
         }
        },
#        {
#            'Area': 20,
#            'Direction' : 'Top',
#            'Name' : 'Roof',
#            'Space' : 'Main',
#            'NextSpace' : 'OutsideAir',
#            'Structure' : 'Wood', 
#            'Type' : 'Ceiling',
#            'UA': 1.0,
#            'InputMethodWood' : 'InputAllLayers',
#            'RoofConstructionMethod' : 'InsuladdBackhorizontal',
#            'Parts' :  {
#                'InsulRafterandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
#                'InsulrafterHeatbridgeadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
#                'InsuladdHeatbridgemember' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
#                'HeatbridgeMemberandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]}
#         }
#        },
        {
            'Area': 30,
            'Direction' : 'North',
            'Name' : 'Wall',
            'Space' : 'Main',
            'Structure' : 'Wood', 
            'NextSpace' : 'OutsideAir',
            'Type' : 'Wall',
            'UA': 1.0,
            'InputMethodWood' : 'InputAllLayers',
            'WallConstructionMethod' : 'WallInsuladdBackvertical',
            'Parts' :  {
                'WallInsulFilledandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallInsulfilledHeatbridgeadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallInsuladdHeatbridgemember' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallInsuladdHeatbridgelintel' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallHeatbridgeMemberandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
                'WallHeatbridgeLintelandadd' : {'Layers' : [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]} 
         }
        }
    ]
}

d_rf = []
for x in d['Walls'] :
    d_rf.extend([get_responce_factor(d['Common'], x)])
    
print(integrate_responce_factor(d_rf))

Importing Jupyter notebook from ConvertUValueToSpec.ipynb
Importing Jupyter notebook from C:\Users\serikawa\Documents\e7_inputdataconvert\heatload_calc\Wall.ipynb
{'Main': {'Bottom': {'Area': 40, 'ResponceFactor': {'RFT0': 0.53268507142005506, 'RFT1': [3.4048665392638229e-05, -0.00059537072387952231, 0.0057474896938861948, -0.041652075233001892, 0.43228112052116369, 0.03422879200329626, -0.012257398728805661, 0.0010447564185243311], 'RFA0': 0.83729902951300184, 'RFA1': [4.9206359654288484e-06, -8.7787933883761574e-05, 0.0008841459137036579, -0.0069901659758621493, 0.086946662035929198, 0.0102863801836922, 0.01816992577373926, 0.0010253752382646376]}}, 'North': {'Area': 30, 'ResponceFactor': {'RFT0': 0.80337149603757385, 'RFT1': [-5.869868434482868e-06, 0.00010113265583475818, -0.00093923403651378467, 0.0059925729600130064, -0.021619673475672476, 0.23605567287610038, -0.025050637433720706, 0.00086734351357672826], 'RFA0': 0.83802665499624696, 'RFA1': [-5.4994019493980017e-06, 9.25596391